In [ ]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
from gensim.models import Word2Vec
import multiprocessing
import numpy as np
import random
import sys
import tensorflow as tf
from tensorflow.keras.layers import  Dense, Dropout, Activation, LSTM, Bidirectional
from tensorflow.keras import Model, Sequential 
import numpy as np
from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
z='/content/drive/My Drive/combine.txt'
corpus=[]
with open(z,'r',encoding='utf-8') as file:
      # reading each line     
    for line in file:         
        for word in line.split():           
            corpus.append(word)
print(corpus[4])
emb_dim=300
wv= Word2Vec(corpus, size=emb_dim, sg=1, window=5, min_count=5, negative=15, iter=10, workers=multiprocessing.cpu_count())

आपका


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
text_in_words=corpus


In [ ]:
MIN_WORD_FREQUENCY=20
word_freq = {}
for word in text_in_words:
    word_freq[word] = word_freq.get(word, 0) + 1

ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)

words = set(text_in_words)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

Unique words before ignoring: 58568
Ignoring words with frequency < 20
Unique words after ignoring: 5812


In [ ]:
SEQUENCE_LEN=30
STEP = 1
sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

Ignored sequences: 1469726
Remaining sequences: 357619


In [ ]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)

In [ ]:
(sentences, next_words), (sentences_test, next_words_test) = shuffle_and_split_training_set(sentences, next_words)

Shuffling sentences
Size of training set = 350466
Size of test set = 7153


In [ ]:
dropout=0.2
model = Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(words), output_dim=1024))
model.add(Bidirectional(LSTM(128)))
if dropout > 0:
    model.add(Dropout(dropout))
model.add(Dense(len(words)))
model.add(Activation('softmax'))

In [ ]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print('\n----- Generating text after Epoch: %d\n' % epoch)
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]
    print(seed)
   # seed = sent.split(' ')

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        print('----- Diversity:' + str(diversity) + '\n')
        print('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        print(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN))
            for t, word in enumerate(sentence):
                x_pred[0, t] = word_indices[word]

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            print(" "+next_word,end=' ')
    print('='*80 + '\n')

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
                "loss{loss:.4f}-accuracy{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}" % \
                (len(words), SEQUENCE_LEN, MIN_WORD_FREQUENCY)
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)
callbacks_list = [checkpoint, print_callback, early_stopping]



In [ ]:
BATCH_SIZE=32
STEP=1
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
examples = sys.argv[2]
examples_file = open(examples, "w")
model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
    steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
    epochs=8,
    callbacks=callbacks_list,
    validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),              
    validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)
model.save_weights("/content/drive/My Drive/model.h5")

Epoch 1/8
10953/10953 [==============================] - ETA: 0s - loss: 3.9069 - accuracy: 0.4002WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./checkpoints/LSTM_LYRICS-epoch001-words5812-sequence30-minfreq20-loss3.9069-accuracy0.4002-val_loss3.2911-val_acc0.4651/assets

----- Generating text after Epoch: 0

['ये', 'आप', 'सब', 'जानते', 'हैं।', 'गरीब', 'तो', 'सोच', 'ही', 'नहीं', 'सकता', 'था', 'कि', 'उसके', 'घर', 'में', 'गैस', 'का', 'सिलेंडर', 'हो', 'सकता', 'है।', 'वो', 'तो', 'बेचारा', 'हिम्मत', 'ही', 'नहीं', 'करता', 'था।']
----- Diversity:0.3

----- Generating with seed:
"ये आप सब जानते हैं। गरीब तो सोच ही नहीं सकता था कि उसके घर में गैस का सिलें

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 other  countries  in  this  regard.  We  are  keen  to  take  the  conservation  of  migratory  birds  to  a ----- Diversity:0.4

----- Generating with seed:
"Presidency, India would steer in the following areas: India is a part of the Central Asian Flyway for migratory birds. With a view to conserve the birds along the Central"

Presidency, India would steer in the following areas: India is a part of the Central Asian Flyway for migratory birds. With a view to conserve the birds along the Central
 Asian  Flyway  and  their  habitats,  India  has  prepared  a  ‘National  Action  Plan  for  conservation  of  Migratory  Birds  along  the  Central  Asian  Flyway’.  India  would  be  happy  to  facilitate  preparation  of  Action  Plans  for  other  countries  in  this  regard.  We  are  keen  to  take  the  conservation  of  migratory  birds  to  a ----- Diversity:0.5

----- Generating with seed:
"Presidency, India would steer in the following areas: India is a part of the Central Asian 

In [ ]:
model.load_weights("/content/drive/My Drive/model.h5")

In [ ]:
def run(k):
    seed =k
    print(seed)
   # seed = sent.split(' ')

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        print('----- Diversity:' + str(diversity) + '\n')
        print('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        print(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN))
            for t, word in enumerate(sentence):
                x_pred[0, t] = word_indices[word]

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            print(" "+next_word,end=' ')
    print('='*80 + '\n')

In [ ]:
run(['जब', 'देश', 'में', 'माताएं-बहनें', 'पंचायतों', 'और', 'शहरों', 'की'])

['जब', 'देश', 'में', 'माताएं-बहनें', 'पंचायतों', 'और', 'शहरों', 'की']
----- Diversity:0.3

----- Generating with seed:
"जब देश में माताएं-बहनें पंचायतों और शहरों की"

जब देश में माताएं-बहनें पंचायतों और शहरों की
 जयंती  जाए।  इसी  जाए।  लेकिन  गए।  क्यों  पड़े  इसी  ये  ये  ये  जाए।  यानी  जाए।  ये  हों।  अगस्त  ये  ये  ये  रही  ये  जाए।  रही  होगा।  ये  होगा।  भाइयो-बहनो,  नहीं  ये  जाए।  जाए।  जाए।  जाए।  जाए।  भाइयो-बहनो,  जाए।  ये  जाए।  जाए।  जाए।  जाए।  ये  अगस्त  ये  जाए।  ये  जाए।  जाए। ----- Diversity:0.4

----- Generating with seed:
"जब देश में माताएं-बहनें पंचायतों और शहरों की"

जब देश में माताएं-बहनें पंचायतों और शहरों की
 जाए।  जाए।  ये  जाए।  लेकिन  गए।  जाए।  जाए।  जाए।  ये  जाए।  जाए।  अरे  नहीं  ये  ये  जाए।  जाए।  जाए।  ये  जाए।  जाए।  अगस्त  ये  ये  जाए।  ये  जयंती  रही  नहीं  जाए।  नहीं  ये  ये  जयंती  लेकिन  जाए।  हो  भाइयो-बहनो,  पड़े  जाए।  ये  नहीं  जाए।  जाए।  नहीं  जाए,  हां  जाए।  ये ----- Diversity:0.5

----- Generating with seed:
"जब देश में माताएं-बहनें पं